# Limpieza de los Datos

## Verificación de Datos

In [1]:
import pandas as pd

df = pd.read_excel('../data/raw/online_retail_II.xlsx')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


Se verifica la entrada de datos, tipos de datos y valores faltantes, anteriormente se realizo en la exploracion inicial de datos, pero se vuelve a realizar para verificar y asegurar que los datos esten correctos antes de proceder con la limpieza.

##  Estandarización y Optimización de Tipos de Datos

In [3]:
df = df.assign(
    Invoice=df["Invoice"].astype("string"),
    StockCode=df["StockCode"].astype("string"),
    Description=df["Description"].astype("string"),
    Quantity=df["Quantity"].astype("int16"),
    Price=df["Price"].astype("float32"),
    Country=df["Country"].astype("string")
) # metodo assign para reasignar tipos de datos (recorre cada columna y le asigna el nuevo tipo de dato)

df["Customer ID"] = df["Customer ID"].astype("Int32") # se asigna el tipo de dato Int32 para permitir valores nulos
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"]) # convertir a datetime


Se realizó la conversión explícita de los tipos de datos con el objetivo de optimizar rendimiento, memoria y consistencia del dataset para análisis posteriores.

Las columnas categoricas o textuales como, `'Invoice'`, `'StockCode'`, `'Description'`, `'Country'` fueron convertidas a tipo `'string'` para optimizar almacenamiento y evitar ambiguedades con el tipo `'object'` para garantizar consistencia en el manejo de datos en el formato parquet.

Columnas numericas discretas como `'Quantity'` se convirtieron a tipo `'int32'` para reducir el uso de memoria y mejorar el rendimiento en cálculos y análisis estadísticos.

Columnas numericas continuas como `'Price'` se convirtieron a tipo `'float32'` para optimizar el uso de memoria frente a `'float64'` manteniendo la precisión necesaria para análisis financieros y cálculos de valor monetario.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  string        
 1   StockCode    525461 non-null  string        
 2   Description  522533 non-null  string        
 3   Quantity     525461 non-null  int16         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float32       
 6   Customer ID  417534 non-null  Int32         
 7   Country      525461 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 25.6 MB


Se observa la reducción significativa en el uso de memoria del DataFrame después de cambiar los tipos de datos:
- **Memoria original**: 32.1+ MB
- **Memoria optimizada**: 25.6 MB

La optimización se logró mediante la conversión de columnas a tipos de datos más eficientes:
- Columnas de object a tipo `string`
- `Quantity` a `int16` (rango suficiente para valores esperados)
- `Price` a `float32` (precisión adecuada para precios)
- `Customer ID` a `Int32` (permite valores nulos)

Esta optimización mejora el rendimiento en operaciones posteriores y reduce el consumo de recursos del sistema.

## Estandarización del Formato de los Datos

In [5]:
df.to_parquet(
    "../data/intermedios/online_retail_II.parquet",
    engine="pyarrow",
    compression="snappy"
)

In [6]:
df_parquet = pd.read_parquet("../data/intermedios/online_retail_II.parquet")

Se guarda el DataFrame limpio en un archivo parquet debido a su eficiencia en almacenamiento y velocidad de lectura/escritura, especialmente para grandes volúmenes de datos. Parquet utiliza compresión y codificación optimizadas, lo que reduce el espacio en disco y mejora el rendimiento en operaciones de análisis de datos. 

Este formato (parquet) se utilizo debido a la cantidad de datos que se esta manejando en el proyecto, facilitando y ahorrando tiempo en futuras cargas y manipulaciones del DataFrame.

## Eliminación de Valores Nulos

In [7]:
df_parquet.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Se observa una gran cantidad de valores nulos en la columna `'Customer ID'`, lo cual es crítico para el análisis de CLTV, ya que esta columna es esencial para identificar a los clientes. Además, la columna `'Description'` también presenta valores nulos.

In [8]:
df_parquet = df_parquet.dropna(subset=['Customer ID', 'Description'])
df_parquet.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

Se eliminan valores nulos en las columnas 'Customer ID' y 'Description' ya que son los unicos campos que tienen valores nulos y son importantes para el analisis.

In [9]:
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  string        
 1   StockCode    417534 non-null  string        
 2   Description  417534 non-null  string        
 3   Quantity     417534 non-null  int16         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float32       
 6   Customer ID  417534 non-null  Int32         
 7   Country      417534 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 23.5 MB


Despues de la eliminacion de valores nulos, el DataFrame resultante no contiene valores nulos en ninguna columna, asegurando la integridad de los datos, crucial para análisis posteriores y modelado predictivo.

## Eliminación de Valores Duplicados

In [10]:
df_parquet.duplicated().sum()# Verificar que no hay duplicados

np.int64(6771)

Se verifica la presencia de valores duplicados en el DataFrame y se encuentran 6771 filas duplicadas, que deben ser eliminadas para asegurar la calidad de los datos.

In [11]:
df_parquet = df_parquet.drop_duplicates() # Eliminar duplicados
df_parquet.duplicated().sum()

np.int64(0)

In [12]:
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410763 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      410763 non-null  string        
 1   StockCode    410763 non-null  string        
 2   Description  410763 non-null  string        
 3   Quantity     410763 non-null  int16         
 4   InvoiceDate  410763 non-null  datetime64[ns]
 5   Price        410763 non-null  float32       
 6   Customer ID  410763 non-null  Int32         
 7   Country      410763 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 23.1 MB


Despues de la eliminacion de valores nulos y duplicados, el DataFrame de `'525461'` registros paso a tener `'410763'` registros, lo que representa una reduccion significativa en el tamaño del dataset y mejora la calidad de los datos.

## Verificación y Eliminación de Valores Inválidos

In [23]:
count_quantity = (df_parquet['Quantity'] == 0).sum() # Contar cuantos registros tienen una cantidad igual a 0.
count_price = (df_parquet['Price'] <= 0).sum() # Contar cuantos registros tienen un precio menor o igual a 0.

print(f"Quantity: {count_quantity}")
print(f"Price: {count_price}")

Quantity: 0
Price: 0


En la verificacion de valores inválidos, se encontraron 31 valores negativos de la variable `'Price'`, lo que podria indicar errores de entrada de datos, regalos o promociones, estos valores se eliminaran ya que no son necesarios para el analisis del CLTV, precios negativos o con 0 no aportan valor al analisis y pueden distorsionar los resultados.

En cuanto a la variable `'Quantity'`, se verifican únicamente los registros con cantidad igual a 0, los cuales pueden representar promociones o errores de entrada de datos, por lo que no aportan información relevante al análisis.
Los valores negativos de `'Quantity'` se consideran útiles, ya que indican devoluciones de productos. Estas devoluciones permiten ajustar el valor económico neto por cliente a nivel agregado, por lo que no se eliminan del dataset.

In [24]:
df_parquet = df_parquet[
    (df_parquet['Quantity'] != 0) &
    (df_parquet['Price'] > 0)
] # Contar cuantos registros tienen un precio menor o igual a 0.


print(f"Quantity: {(df_parquet['Quantity'] == 0).sum()}")
print(f"Price: {(df_parquet['Price'] <= 0).sum()}")

Quantity: 0
Price: 0


Luego de eliminar los registros se verifica nuevamente la presencia de valores inválidos en las columnas `'Price'` y `'Quantity'`, confirmando que no existen valores negativos o cero en `'Price'` y que los valores en `'Quantity'` son adecuados para el análisis.

## Eliminacion de Facturas con estado 'C' (Canceladas)

In [39]:
print(f"Facturas en estado canceladas: {df_parquet['Invoice'].str.startswith('C').sum()}")

df_parquet = df_parquet[~df_parquet['Invoice'].str.startswith('C')] # Eliminar facturas canceladas

Facturas en estado canceladas: 0


In [40]:
print(f"Facturas en estado canceladas despues de eliminar: {df_parquet['Invoice'].str.startswith('C').sum()}")

Facturas en estado canceladas despues de eliminar: 0


Segun la documentacion del dataset, las facturas con estado 'C' indican que han sido canceladas. Entonces de acuerdo a esto, se verifica la existencia de facturas con estado 'C' (Canceladas) en la columna `'Invoice'` y se procede a eliminarlas del DataFrame para asegurar que solo se analicen transacciones que aporten valor real al análisis de CLTV.

## Normalización y estandarización de nombres de columnas

In [41]:
df_parquet.columns = (
    df_parquet.columns.str.strip()
            .str.lower()
            .str.replace('á', 'a')
            .str.replace('é', 'e')
            .str.replace('í', 'i')
            .str.replace('ó', 'o')
            .str.replace('ú', 'u')
            .str.replace('ñ', 'n')
            .str.replace(' ', '_')
)

df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400916 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      400916 non-null  string        
 1   stockcode    400916 non-null  string        
 2   description  400916 non-null  string        
 3   quantity     400916 non-null  int16         
 4   invoicedate  400916 non-null  datetime64[ns]
 5   price        400916 non-null  float32       
 6   customer_id  400916 non-null  Int32         
 7   country      400916 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 22.6 MB


Se normalizaron los nombres de las columnas con el fin de estandarizar su formato y facilitar el manejo del dataset durante el análisis. Este proceso permite evitar errores al manipular los datos, mejora la legibilidad del código y asegura consistencia en las operaciones posteriores, especialmente durante el análisis exploratorio y la construcción de variables para el cálculo del CLTV.

## Tratamiento de valores atípicos (outliers)

Los valores atípicos observados en la etapa anterior (Exploracion inicial) corresponden a comportamientos reales del negocio, propios de una tienda online, como compras de alto volumen o clientes de alto valor. Dado que el objetivo del análisis es el cálculo del CLTV, estos outliers representan información relevante y no errores de datos, por lo que no se aplicaron técnicas de eliminación o recorte. Se usaran tecnicas de modelado robustas que puedan manejar estos valores atípicos sin afectar negativamente el rendimiento del modelo.

## Exportación del DataFrame Limpio

In [ ]:
df.to_excel('../data/processed/online_retail_II_cleaned.xlsx', index=False)